# Skills Matching (Pages → DB Compétences)

- Charge le dernier CSV d'extraction dans `output/*with_content*.csv`.
- Nettoie et normalise le texte extrait.
- Applique des regex et règles pour identifier les compétences techniques.
- Sauvegarde les résultats dans `results/skills_extracted_YYYYMMDD_HHMMSS.csv`.

In [ ]:
import os
import re
import pandas as pd
import numpy as np
from pathlib import Path
import datetime as dt
from collections import Counter

print("Imports réalisés avec succès")

In [ ]:
# Configuration
OUTPUT_DIR = Path('output')
RESULTS_DIR = Path('results')
RESULTS_DIR.mkdir(exist_ok=True)

print(f"Dossier de résultats: {RESULTS_DIR}")
print(f"Dossier d'entrée: {OUTPUT_DIR}")

## 1. Définition des compétences à rechercher

In [ ]:
# Base de données des compétences techniques
COMPETENCES_TECHNIQUES = {
    'Langages de programmation': [
        'python', 'java', 'javascript', 'typescript', 'c#', 'c++', 'c', 'php', 'ruby', 'go', 'rust',
        'swift', 'kotlin', 'scala', 'r', 'matlab', 'sql', 'html', 'css', 'sass', 'less'
    ],
    'Frameworks et bibliothèques': [
        'react', 'angular', 'vue', 'node.js', 'express', 'django', 'flask', 'spring', 'laravel',
        'symfony', 'rails', 'asp.net', '.net', 'bootstrap', 'jquery', 'tensorflow', 'pytorch',
        'scikit-learn', 'pandas', 'numpy'
    ],
    'Bases de données': [
        'mysql', 'postgresql', 'mongodb', 'redis', 'elasticsearch', 'sqlite', 'oracle',
        'sql server', 'cassandra', 'neo4j', 'firebase'
    ],
    'Cloud et DevOps': [
        'aws', 'azure', 'gcp', 'docker', 'kubernetes', 'jenkins', 'gitlab ci', 'github actions',
        'terraform', 'ansible', 'vagrant', 'linux', 'ubuntu', 'centos', 'nginx', 'apache'
    ],
    'Outils et technologies': [
        'git', 'svn', 'jira', 'confluence', 'slack', 'teams', 'figma', 'sketch', 'photoshop',
        'illustrator', 'after effects', 'premiere pro', 'autocad', 'solidworks'
    ],
    'Méthodologies': [
        'agile', 'scrum', 'kanban', 'devops', 'ci/cd', 'tdd', 'bdd', 'clean code',
        'design patterns', 'microservices', 'api rest', 'graphql'
    ]
}

# Compétences transversales
COMPETENCES_TRANSVERSALES = {
    'Communication': [
        'communication', 'présentation', 'rédaction', 'négociation', 'relation client',
        'animation d\'équipe', 'formation', 'pédagogie'
    ],
    'Gestion de projet': [
        'gestion de projet', 'planning', 'budget', 'coordination', 'organisation',
        'suivi', 'reporting', 'analyse des risques'
    ],
    'Langues': [
        'anglais', 'allemand', 'espagnol', 'italien', 'mandarin', 'arabe', 'russe',
        'bilingue', 'trilingue', 'niveau b2', 'niveau c1', 'toeic', 'toefl'
    ]
}

# Fusionner toutes les compétences
TOUTES_COMPETENCES = {**COMPETENCES_TECHNIQUES, **COMPETENCES_TRANSVERSALES}

print(f"Nombre de catégories de compétences: {len(TOUTES_COMPETENCES)}")
total_skills = sum(len(skills) for skills in TOUTES_COMPETENCES.values())
print(f"Nombre total de compétences définies: {total_skills}")

## 2. Fonctions utilitaires

In [ ]:
def nettoyer_texte(texte):
    """Nettoie et normalise le texte pour l'extraction de compétences."""
    if pd.isna(texte) or not isinstance(texte, str):
        return ""
    
    # Convertir en minuscules
    texte = texte.lower()
    
    # Remplacer les caractères spéciaux par des espaces
    texte = re.sub(r'[^\\w\\s./-]', ' ', texte)
    
    # Normaliser les espaces multiples
    texte = re.sub(r'\\s+', ' ', texte)
    
    return texte.strip()

def extraire_competences(texte, competences_dict):
    """Extrait les compétences trouvées dans le texte."""
    if not texte:
        return {}
    
    competences_trouvees = {}
    
    for categorie, skills_list in competences_dict.items():
        competences_trouvees[categorie] = []
        
        for skill in skills_list:
            # Recherche avec limites de mots pour éviter les faux positifs
            pattern = r'\\b' + re.escape(skill.lower()) + r'\\b'
            if re.search(pattern, texte):
                competences_trouvees[categorie].append(skill)
    
    # Supprimer les catégories vides
    competences_trouvees = {k: v for k, v in competences_trouvees.items() if v}
    
    return competences_trouvees

def compter_competences(liste_competences):
    """Compte les occurrences de chaque compétence."""
    compteur = Counter()
    
    for competences_doc in liste_competences:
        for categorie, skills in competences_doc.items():
            for skill in skills:
                compteur[skill] += 1
    
    return compteur

## 3. Chargement des données

In [ ]:
# Trouver le fichier CSV le plus récent
csv_files = list(OUTPUT_DIR.glob('*with_content*.csv'))

if not csv_files:
    raise FileNotFoundError(f"Aucun fichier CSV trouvé dans {OUTPUT_DIR}")

# Prendre le plus récent
latest_csv = sorted(csv_files)[-1]
print(f"Chargement du fichier: {latest_csv}")

# Charger les données
df = pd.read_csv(latest_csv)
print(f"Nombre de documents chargés: {len(df)}")
print(f"Colonnes disponibles: {list(df.columns)}")

# Identifier la colonne de texte
if 'extracted_text' in df.columns:
    text_column = 'extracted_text'
elif 'content' in df.columns:
    text_column = 'content'
else:
    text_column = df.columns[-1]  # Dernière colonne par défaut

print(f"Colonne de texte utilisée: {text_column}")
print(f"Documents avec du texte: {df[text_column].notna().sum()}")

## 4. Nettoyage et préparation

In [ ]:
# Nettoyer le texte
print("Nettoyage du texte...")
df['texte_nettoye'] = df[text_column].apply(nettoyer_texte)

# Filtrer les documents vides
df_clean = df[df['texte_nettoye'].str.len() > 10].copy()
print(f"Documents après nettoyage: {len(df_clean)}")

# Statistiques sur la longueur du texte
df_clean['longueur_texte'] = df_clean['texte_nettoye'].str.len()
print(f"Longueur moyenne du texte: {df_clean['longueur_texte'].mean():.0f} caractères")
print(f"Longueur médiane du texte: {df_clean['longueur_texte'].median():.0f} caractères")

## 5. Extraction des compétences

In [ ]:
# Extraire les compétences pour chaque document
print("Extraction des compétences...")

competences_extraites = []
for idx, row in df_clean.iterrows():
    competences = extraire_competences(row['texte_nettoye'], TOUTES_COMPETENCES)
    competences_extraites.append(competences)

df_clean['competences'] = competences_extraites

# Compter le nombre de compétences par document
df_clean['nb_competences'] = df_clean['competences'].apply(
    lambda x: sum(len(skills) for skills in x.values())
)

print(f"Documents avec au moins une compétence: {(df_clean['nb_competences'] > 0).sum()}")
print(f"Nombre moyen de compétences par document: {df_clean['nb_competences'].mean():.2f}")
print(f"Maximum de compétences dans un document: {df_clean['nb_competences'].max()}")

## 6. Analyse et statistiques

In [ ]:
# Compter les compétences les plus fréquentes
compteur_competences = compter_competences(competences_extraites)
competences_populaires = compteur_competences.most_common(20)

print("Top 20 des compétences les plus fréquentes:")
for i, (skill, count) in enumerate(competences_populaires, 1):
    print(f"{i:2d}. {skill:<20} : {count:3d} occurrences")

In [ ]:
# Statistiques par catégorie
stats_categories = {}

for categorie in TOUTES_COMPETENCES.keys():
    # Compter les documents ayant au moins une compétence de cette catégorie
    docs_avec_categorie = sum(1 for comp in competences_extraites if categorie in comp and comp[categorie])
    
    # Compter le total des compétences de cette catégorie
    total_competences = sum(len(comp.get(categorie, [])) for comp in competences_extraites)
    
    stats_categories[categorie] = {
        'documents': docs_avec_categorie,
        'total_competences': total_competences,
        'pourcentage_docs': (docs_avec_categorie / len(df_clean)) * 100
    }

print("\\nStatistiques par catégorie de compétences:")
print(f"{'Catégorie':<25} {'Docs':<6} {'Total':<6} {'% Docs':<8}")
print("-" * 50)

for categorie, stats in stats_categories.items():
    print(f"{categorie:<25} {stats['documents']:<6} {stats['total_competences']:<6} {stats['pourcentage_docs']:<6.1f}%")

## 7. Préparation des résultats pour export

In [ ]:
# Préparer le DataFrame de résultats
resultats = []

for idx, row in df_clean.iterrows():
    doc_info = {
        'filename': row.get('filename', f'doc_{idx}'),
        'nb_competences_total': row['nb_competences'],
        'longueur_texte': row['longueur_texte']
    }
    
    # Ajouter les compétences par catégorie
    for categorie in TOUTES_COMPETENCES.keys():
        competences_cat = row['competences'].get(categorie, [])
        doc_info[f'nb_{categorie.lower().replace(" ", "_")}'] = len(competences_cat)
        doc_info[f'{categorie.lower().replace(" ", "_")}_liste'] = ', '.join(competences_cat)
    
    # Ajouter d'autres métadonnées si disponibles
    for col in ['page_count', 'extraction_time']:
        if col in row:
            doc_info[col] = row[col]
    
    resultats.append(doc_info)

df_resultats = pd.DataFrame(resultats)
print(f"DataFrame de résultats créé avec {len(df_resultats)} lignes et {len(df_resultats.columns)} colonnes")

## 8. Sauvegarde des résultats

In [ ]:
# Créer le nom de fichier avec timestamp
timestamp = dt.datetime.now().strftime('%Y%m%d_%H%M%S')
fichier_resultats = RESULTS_DIR / f'skills_extracted_{timestamp}.csv'

# Sauvegarder
df_resultats.to_csv(fichier_resultats, index=False, encoding='utf-8')
print(f"Résultats sauvegardés dans: {fichier_resultats}")

# Sauvegarder aussi un résumé des statistiques
fichier_stats = RESULTS_DIR / f'skills_stats_{timestamp}.json'
import json

resume_stats = {
    'timestamp': timestamp,
    'source_file': str(latest_csv),
    'total_documents': len(df_clean),
    'documents_avec_competences': (df_clean['nb_competences'] > 0).sum(),
    'competences_les_plus_frequentes': dict(competences_populaires[:10]),
    'stats_par_categorie': stats_categories
}

with open(fichier_stats, 'w', encoding='utf-8') as f:
    json.dump(resume_stats, f, indent=2, ensure_ascii=False)

print(f"Statistiques sauvegardées dans: {fichier_stats}")
print("\\nExtraction des compétences terminée avec succès !")

## 9. Aperçu des résultats

In [ ]:
# Afficher un aperçu des résultats
print("Aperçu des premiers résultats:")
print(df_resultats.head())

print("\\nColonnes disponibles dans les résultats:")
for i, col in enumerate(df_resultats.columns, 1):
    print(f"{i:2d}. {col}")

print(f"\\nNombre total de documents traités: {len(df_resultats)}")
print(f"Taille du fichier de résultats: {fichier_resultats.stat().st_size / 1024:.1f} KB")